The notebook contains examples of loading LFP signals from .npy files and calculating power spectra using methods from scipy and neurodsp packages. 

It plots an example LFP trace (2 seconds of the signal) and power spectrum for three areas. Plots are saved in the figures google drive folder.

In [1]:
import numpy as np
from scipy.signal import welch
from neurodsp.spectral import compute_spectrum
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

#### Parameters

In [2]:
# your path to the dataset
dataset_folder = Path('D:/bernstein/dataset_py/')

mouse_id = 1
fs = 500  # Hz, sampling rate
lfp_files = [f"lfp_1.npy", f"lfp_2.npy", f"lfp_3.npy"]
lfp_folder = dataset_folder / str(mouse_id)

#### Loading LFPs

In [3]:
# Loading the data
lfps = [np.load(lfp_folder / f) for f in lfp_files]

for i, lfp in enumerate(lfps, start=1):
    print(f"LFP{i}: shape {lfp.shape}, duration {lfp.shape[1]/fs:.1f} sec ({lfp.shape[1]/fs/60:.1f} min), "
          f"channels {lfp.shape[0]}")

LFP1: shape (28, 1675486), duration 3351.0 sec (55.8 min), channels 28
LFP2: shape (25, 1675486), duration 3351.0 sec (55.8 min), channels 25
LFP3: shape (47, 1675486), duration 3351.0 sec (55.8 min), channels 47


In [8]:
# Plot a small segment of raw LFPs (from the first channel of 3 brain areas)
t = np.arange(0, 2*fs) / fs  # first 2 seconds
fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
for i, (ax, lfp) in enumerate(zip(axes, lfps), start=1):
    ax.plot(t, lfp[0, :2*fs], lw=0.8, c='k')  # first channel
    ax.set_title(f"Brain area {i}, channel 1")
    ax.set_ylabel("LFP")
axes[-1].set_xlabel("Time (s)")
sns.despine()

fig.savefig(dataset_folder / "output/lfp_signals_example.png", bbox_inches="tight")

#### Calculating power spectra

In [5]:
# Compute power spectra (median LFP per area), with welch from scipy and compute_spectrum from neurodsp
spectra = {}
for fname, lfp in zip(lfp_files, lfps):
    area = fname.replace(".npy", "")  
    sig = np.median(lfp, axis=0) # median across channels

    # Welch 
    freqs_welch, psd_welch = welch(sig, fs=fs, nperseg=fs*2)

    # NeuroDSP 
    freqs_ndsp, psd_ndsp = compute_spectrum(sig, fs, method='welch', nperseg=fs*2)

    spectra[area] = {
        "welch": (freqs_welch, psd_welch),
        "neurodsp": (freqs_ndsp, psd_ndsp),
    }

In [18]:
# Plot spectra only 
colors = ['#0072B2', '#E69F00', '#000000']
plot_loglog = False # plotting in loglog scale

fig, ax = plt.subplots(1, 2, figsize=(10,4))
for (area, spec), color in zip(spectra.items(), colors):
    freqs, psd = spec["neurodsp"]
    if plot_loglog:
        ax[0].loglog(freqs, psd, label=area, color=color) 
    else:
        ax[0].semilogy(freqs, psd, label=area, color=color)

for (area, spec), color in zip(spectra.items(), colors):
    freqs, psd = spec["welch"]
    if plot_loglog:
        ax[1].loglog(freqs, psd, label=area, color=color) 
    else:   
        ax[1].semilogy(freqs, psd, label=area, color=color)

for axes in ax.flat:
    axes.set_xlabel("Frequency (Hz)")
    axes.set_ylabel("Power")
    axes.legend(frameon=False)
    
ax[0].set_title("Median LFP power spectra (neuroDSP)")
ax[1].set_title("Median LFP power spectra (scipy)")

plt.tight_layout()

sns.despine()

fig.savefig(dataset_folder / "output/power_spectra_example.png", bbox_inches="tight")